In [100]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
from datetime import datetime

In [140]:
# 1rm for 5x5 = Max = ((weight X reps) X 0.0333) + weight

base_url = 'https://www.jefit.com/members/user-logs/log/?xid='
user_id = '3806546'
specific_date = '2020-02-05' # Must be in YYYY-MM-DD

request_url = base_url + user_id + '&dd=' + specific_date
print(request_url)

https://www.jefit.com/members/user-logs/log/?xid=3806546&dd=2020-02-05


In [133]:
def date_list_maker(year):
    
    date_list = []

    for month in range(1, 13):
        for day in range(1, 32):
            try:
                date_list.append(datetime(year, month, day).strftime("%Y-%m-%d"))
                #print(datetime(2020, month, day).strftime("%Y-%m-%d"))
            except ValueError:
                pass
    return date_list

In [137]:
date_list_2016 = date_list_maker(2016)
date_list_2017 = date_list_maker(2017)
date_list_2018 = date_list_maker(2018)
date_list_2019 = date_list_maker(2019)

In [139]:
date_list_2018

['2018-01-01',
 '2018-01-02',
 '2018-01-03',
 '2018-01-04',
 '2018-01-05',
 '2018-01-06',
 '2018-01-07',
 '2018-01-08',
 '2018-01-09',
 '2018-01-10',
 '2018-01-11',
 '2018-01-12',
 '2018-01-13',
 '2018-01-14',
 '2018-01-15',
 '2018-01-16',
 '2018-01-17',
 '2018-01-18',
 '2018-01-19',
 '2018-01-20',
 '2018-01-21',
 '2018-01-22',
 '2018-01-23',
 '2018-01-24',
 '2018-01-25',
 '2018-01-26',
 '2018-01-27',
 '2018-01-28',
 '2018-01-29',
 '2018-01-30',
 '2018-01-31',
 '2018-02-01',
 '2018-02-02',
 '2018-02-03',
 '2018-02-04',
 '2018-02-05',
 '2018-02-06',
 '2018-02-07',
 '2018-02-08',
 '2018-02-09',
 '2018-02-10',
 '2018-02-11',
 '2018-02-12',
 '2018-02-13',
 '2018-02-14',
 '2018-02-15',
 '2018-02-16',
 '2018-02-17',
 '2018-02-18',
 '2018-02-19',
 '2018-02-20',
 '2018-02-21',
 '2018-02-22',
 '2018-02-23',
 '2018-02-24',
 '2018-02-25',
 '2018-02-26',
 '2018-02-27',
 '2018-02-28',
 '2018-03-01',
 '2018-03-02',
 '2018-03-03',
 '2018-03-04',
 '2018-03-05',
 '2018-03-06',
 '2018-03-07',
 '2018-03-

In [141]:
user_workout = requests.get(request_url).text

In [142]:
workout_html_parse = BeautifulSoup(user_workout)

In [143]:
workout_html_parse(class_="exercise-block")

[]

In [ ]:
"""
Fields of interest:
logid
exerciseid
ename - exercise name
date
"""

In [2]:
exercises_only = workout_html_parse(class_="exercise-block") # Returns a list of exercises

NameError: name 'workout_html_parse' is not defined

In [1]:
exercises_only[0]

NameError: name 'exercises_only' is not defined

In [79]:
exercises_only[0].select("input[name='weight']")

[<input class="inputlog decimal" maxlength="6" name="weight" size="3" type="text" value="255"/>,
 <input class="inputlog decimal" maxlength="6" name="weight" size="3" type="text" value="255"/>,
 <input class="inputlog decimal" maxlength="6" name="weight" size="3" type="text" value="255"/>,
 <input class="inputlog decimal" maxlength="6" name="weight" size="3" type="text" value="255"/>,
 <input class="inputlog decimal" maxlength="6" name="weight" size="3" type="text" value="255"/>]

In [92]:
with open('workout_list.csv', 'w', newline ='') as workout_file:
    writer = csv.writer(workout_file, delimiter=',')
    writer.writerow(['date', 'log_id', 'exercise_id', 'exercise_name', 'set_number', 'weight', 'rep'])


In [95]:
def entry_grabber(exercise_elements):
    log_entry_list = []

    for exercise in exercise_elements:
        # Exercise Details
        log_id = exercise.select("input[name='logid']")[0]['value']
        exercise_id = exercise.select("input[name='exerciseid']")[0]['value']
        exercise_name = exercise.select("input[name='ename']")[0]['value']
        log_date = exercise.select("input[name='date']")[0]['value']

        x = 1
        # weight and rep for each exercise
        for weight, rep in zip(exercise.select("input[name='weight']"), exercise.select("input[name='rep']")):

            set_number = x # Specify which set number it is; not hardcoded in a value
            weight_value = weight['value']
            rep_value = rep['value']

            log_entry_list.append([log_date, log_id, exercise_id, exercise_name, set_number, weight_value, rep_value])

            x += 1

    return log_entry_list
    

In [96]:
entry = entry_grabber(exercises_only)

In [97]:
with open('workout_list.csv', 'a', newline='') as workout_file:
    writer=csv.writer(workout_file, delimiter=',')
    for data in entry:
        writer.writerow(data)

In [98]:
df = pd.read_csv('workout_list.csv')

In [99]:
df.head(20)

,date,log_id,exercise_id,exercise_name,set_number,weight,rep
0,2020-02-04,3886,12,Barbell Squat,1,255,5
1,2020-02-04,3886,12,Barbell Squat,2,255,5
2,2020-02-04,3886,12,Barbell Squat,3,255,5
3,2020-02-04,3886,12,Barbell Squat,4,255,5
4,2020-02-04,3886,12,Barbell Squat,5,255,5
5,2020-02-04,3887,132,Leg Press,1,320,10
6,2020-02-04,3887,132,Leg Press,2,320,10
7,2020-02-04,3887,132,Leg Press,3,320,10
8,2020-02-04,3887,132,Leg Press,4,320,10
9,2020-02-04,3888,160,Lying Leg Curls,1,130,10
